#Very Important Spark Learning - BY LEARNING this PROGRAM - WE BECOME A DATA ENGINEER (DATA CURATION DEVELOPER & DATA ANALYST)
Simply say- We are going to learn...
next level of SQL (Spark SQL) + Python Function based programming (Framework of Spark DSL) + Datawarehouse (Datalake+Lakehouse) -> Transformation & Analytics

##**1. Data Munging** - (Cleanup) Process of transforming and mapping data from Raw form into Tidy(usable) format with the intent of making it more appropriate and valuable for a variety of downstream purposes such for further Transformation/Enrichment, Egress/Outbound, analytics, Datascience/AI application & Reporting

**Passive Data Munging** - Data Discovery/Data Exploration/ EDA (Exploratory Data Analytics) (every layers ingestion/transformation/analytics/consumption) - Performing an (Data Exploration) exploratory data analysis of the raw data to identify the attributes and patterns. <br>

**Active Data Munging**
1. Combining Data + Schema Evolution/Merging/Merging (Structuring)
2. Validation, Cleansing, Scrubbing - Cleansing (removal of unwanted datasets), Scrubbing (convert raw to tidy)
3. De Duplication and Levels of Standardization () of Data to make it in a usable format (Dataengineers/consumers)

![Stage1](stage1.png)

###a. Passive Data Munging - 
- Visible - Data Discovery/Data Exploration/ EDA (Exploratory Data Analytics) (every layers ingestion/transformation/analytics/consumption) - Performing an (Data Exploration) exploratory data analysis of the raw data to identify the attributes and patterns.

####1. Visibily/Manually opening the file we found couple of data patterns (Manual Exploratory Data Analysis)
- It is a Structured data with comma seperator (CSV)
- No Header, No comments, footer is there in the data
- Total columns are (seperator + 1)
- Data Quality 
- - Null columns & null rows are there
- - duplicate rows & Duplicate keys
- - format issues are there (age is not in number format eg. 7-7)
- - Uniformity issues (Artist, artist)
- - Number of columns are more or less than the expected
- eg. 4000011,Francis,McNamara,47,Therapist,NewYork & 4000014,Beth,Woodard,65
- - Identification of data type

####2. Programatically lets try to find couple of data patterns applying EDA - passively (without modifying, just for description).


In [0]:
#We have to define Spark Session to enter into Spark application
from pyspark.sql.session import SparkSession
spark=SparkSession.builder.appName("we47_Bread_n_Butter2_Important_Application").getOrCreate()

In [0]:
#Data Exploration programatically
rawdf1=spark.read.csv("/Volumes/we47catalog/we47schema/we47_volume/custsmodified",header=False,inferSchema=True).toDF("id","firstname","lastname","age","profession")
rawdf1.show(20,False)
display(rawdf1.take(20))

In [0]:
#Important passive Munging - EDA of schema/structure functions we can use
rawdf1.printSchema()#I am realizing the id & age columns are having some non numerical values (supposed to be numeric)
print(rawdf1.columns)#I am understanding the column numbers/order and the column names
print(rawdf1.dtypes)#Realizing the datatype of every columns (even we can do programattic column & type identification for dynamic programming)
print(rawdf1.schema)#To identify the structure of the data in the StructType and StructField format

In [0]:
#Important passive Datamunging - EDA data functions we can use
#We identified few patterns on this data
#1. Deduplication of rows and given column(s)
#2. Null values ratio across all columns
#3. Distribution (Dense) of the data across all number columns
#4. Min, Max values
#5. StdDeviation - Standard deviation tells you how much the data varies from the average (mean).
#6. Percentile - Distribution percentage from 0 to 100 in 4 quadrants of 25%
print("actual count of the data",rawdf1.count())
print("de-duplicated record (all columns) count",rawdf1.distinct().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated record (all columns) count",rawdf1.dropDuplicates().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated given cid column count",rawdf1.dropDuplicates(['id']).count())#de duplicate the entire columns of the given  dataframe
display(rawdf1.describe())
display(rawdf1.summary())

###**b. Active Data Munging**
1. Combining Data + Schema Evolution/Merging (Structuring)
2. Validation, Cleansing, Scrubbing - Cleansing (removal of unwanted datasets), Scrubbing (convert raw to tidy)
3. De Duplication and Levels of Standardization () of Data to make it in a usable format (Dataengineers/consumers)

In [0]:
#Questions related to multiple files/paths/sub path handling
#I have data in different filenames in a single/multiple location, i need to read all these data in a df - path=["path1/file1","path1/file2","path2/file3"]
#I have data in single pattern of file names in a single/multiple locations or subfolders, i need to read all these data in a df - path=["path1/","path1/","path2/"], pathGlobFilter="custsm*", recursiveFileLookup=True

#Questions related handling evolving data structure with data ingested in different days/periods - Ans. Schema Evolution
#Evolution is growth over the time (Filesystem level).. Eg. Source is sending data with additional columns week over week in csv format
#1. Read and write in Serialized format( ORC,Parquet)
#2. Read DF with mergeSchema = True

#Questions related handling data from different sources with different related structure in a same day - Ans. Schema Merging/Melting (Dataframe level).. Eg. Source1 is sending custsmodified_NY with 5 columns and Source2 is sending custsmodified TX with 4 columns
#1. Read file1 in DF1, read file2 in DF2
#2. Create DF3 by merging DF1 and DF2 using df1.unionByName(df2,allowMissingColumns=True)

####1. Combining Data + Schema Evolution/Merging (Structuring) - Preliminary Datamunging

In [0]:
#Extraction (Ingestion) methodologies
#1. Single file
struct1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(struct1).csv(path="/Volumes/we47catalog/we47schema/we47_volume/custsmodified")
#2. Multiple files (with different names)
rawdf1=spark.read.schema(struct1).csv(path=["/Volumes/we47catalog/we47schema/we47_volume/custsmodified","/Volumes/we47catalog/we47schema/we47_volume/custsmodifiedsample.txt"])
#3. Multiple files in multiple paths or sub paths
rawdf1=spark.read.schema(struct1).csv(path=["/Volumes/we47catalog/we47schema/we47_volume/","/Volumes/we47catalog/we47schema/we47_volume/we47_dir1"],recursiveFileLookup=True,pathGlobFilter="custsm*")

In [0]:
#VERY VERY IMPORTANT FOR PROACTIVE DRIVING OF INTERVIEW
#Active Data munging...
#When you go for Schema Merging/Melting and Schema Evolution?
#Schema Merging/Melting (unionByName,allowMissingColumns)- If we get multiple files
#Schema Evolution (orc/parquet with mergeSchema) - If no. of columns are keeps added by the source system
#when we know structure of the file already - schema merge/ schema not known earlier  - schema evolution

#COMBINING OR SCHEMA MERGING or SCHEMA MELTING of Data from different sources(Important interview question also as like schema evolution...)
#4. Multiple files with different structure in multiple paths or sub paths
strt1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(strt1).csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/"],recursiveFileLookup=True,pathGlobFilter="custsmodified_N*")
strt2="id string, firstname string, age string, profession string,city string"
rawdf2=spark.read.schema(strt2).csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/"],recursiveFileLookup=True,pathGlobFilter="custsmodified_T*")
display(rawdf1)
display(rawdf2)
rawdf_merged=rawdf1.union(rawdf2)#Use union only if the dataframes are having same columns in the same order with same datatype..
display(rawdf_merged)
#Expected right approach to follow
rawdf_merged=rawdf1.unionByName(rawdf2,allowMissingColumns=True)
display(rawdf_merged)

#Lets try to achieve using schema evolution(but not applicable for this data munging activity of different dataframes available at the same time, we just get the dataframes merged/unioned without writing into serialized format)
strt1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(strt1).csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/"],recursiveFileLookup=True,pathGlobFilter="custsmodified_N*")
strt2="id string, firstname string, age string, profession string,city string"
rawdf2=spark.read.schema(strt2).csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/"],recursiveFileLookup=True,pathGlobFilter="custsmodified_T*")
rawdf1.write.orc("/Volumes/workspace/wd36schema/ingestion_volume/targetorc1/",mode='overwrite')#Useful for future over the time
rawdf2.write.orc("/Volumes/workspace/wd36schema/ingestion_volume/targetorc1/",mode='append')
rawdf_evolved=spark.read.orc("/Volumes/workspace/wd36schema/ingestion_volume/targetorc1/",mergeSchema=True)
display(rawdf_evolved)

Just for the simple learning of schema evolution & schema merging/melting<br>
Schema merging/melting<br>
1,rajeshwari day1(source1)<br>
1,rajeshwari,30 day1(source2)<br>

Schema evolution<br>
1,rajeshwari day1<br>
1,rajeshwari,30 day2<br>

Output is same in both cases...<br>
id,name,age<br>
1,rajeshwari,null<br>
1,null,30<br>

####2. Validation, Cleansing, Scrubbing - Cleansing (removal of unwanted datasets), Scrubbing (convert raw to tidy)

#####Validation - Exploration of data by actively applying cleansing & scrubbing
Identifying and filling gaps & Cleaning data to remove outliers and inaccuracies
Preprocessing, Preparation
Cleansing (removal of unwanted datasets eg. na.drop),
Scrubbing (convert of raw to tidy na.fill or na.replace)

In [0]:
from pyspark.sql.types import *
#Validation by doing cleansing
strt1="id int, firstname string, lastname string, age int, profession string"#applying schema strictly to validate and cleanse the data
strt11=StructType([StructField("id",IntegerType(),True),StructField("firstname",StringType(),True),StructField("lastname",StringType(),True),StructField("age",IntegerType(),True),StructField("profession",StringType(),True)])
dfmethod1=spark.read.schema(strt11).csv("/Volumes/we47catalog/we47schema/we47_volume/custsmodified",mode="PERMISSIVE",header=False)
#method1 - permissive with all rows with respective nulls
#We are making nulls where ever data format mismatch is there (cutting down mud portition from potato - scrubbing)
display(dfmethod1)
print("entire count of data",dfmethod1.count())
print("after scrubbing, count of data",len(dfmethod1.collect()))
#or

#method2 - drop malformed rows
strt1="id int, firstname string, lastname string, age int, profession string"#applying schema strictly to validate and 
dfmethod2=spark.read.schema(strt11).csv("/Volumes/we47catalog/we47schema/we47_volume/custsmodified",mode="dropMalformed",header=False)
#We are removing the entire row, where ever data format mismatch is there (number of columns, data type mismatch) (throwing away the entire potato or some portion of it by - cleansing)
display(dfmethod2)
print("entire count of data",dfmethod2.count())
print("after cleansing, count of data",len(dfmethod2.collect()))

In [0]:
from pyspark.sql.types import *
#If we follow method1 (permissive with strict schema verification) or method2 (drop malformed with strict schema verification)
#Challenges we have in this method1 and 2 is - unknown data loss at column level or at the row level
#method3 best methodology of applying active data munging
dfmethod3=spark.read.csv("/Volumes/we47catalog/we47schema/we47_volume/custsmodified",mode="PERMISSIVE",header=False,inferSchema=True)
#or
print(dfmethod3.schema)
strt11=StructType([StructField("id",StringType(),True),StructField("firstname",StringType(),True),StructField("lastname",StringType(),True),StructField("age",StringType(),True),StructField("profession",StringType(),True)])
dfmethod3=spark.read.schema(strt11).csv("/Volumes/we47catalog/we47schema/we47_volume/custsmodified",mode="PERMISSIVE",header=False)
#Validation by doing cleansing (not at the time of creating Dataframe, rather we will clean and scrub subsequently)...
display(dfmethod3)#We are going to validate the raw data without any scrubbing or cleansing applied, we will decide the cleansing/scrubbing strategy in the below stages...
print("entire count of data",dfmethod3.count())
print("after cleansing, count of data",len(dfmethod3.collect()))

#####Rejection Strategy

In [0]:
#Before actively Cleansing or Scrubbing - We have to create a Rejection Strategy to reduce data challenges in the future
strt11=StructType([StructField("id",IntegerType(),True),StructField("firstname",StringType(),True),StructField("lastname",StringType(),True),StructField("age",IntegerType(),True),StructField("profession",StringType(),True),StructField("corruptdata",StringType())])
dfmethod3=spark.read.schema(strt11).csv("/Volumes/we47catalog/we47schema/we47_volume/custsmodified",mode="PERMISSIVE",header=False,columnNameOfCorruptRecord="corruptdata")
display(dfmethod3)
print("entire count of data",dfmethod3.count())
df_reject=dfmethod3.where("corruptdata is not null")
df_reject.drop("corruptdata").write.mode("overwrite").csv("/Volumes/workspace/wd36schema/ingestion_volume/rejects/",mode="overwrite")
print("Data to reject or update the source",len(dfmethod3.where("corruptdata is not null").collect()))

#####Cleansing
It is a process of cleaning/removing or making the data more clean Eg. Cutting/removing debris portion of the potato

In [0]:
#We already know how to do cleansing applying the strict Structure on method1 and method2, but we can do cleansing and scrubbing in a controlled fashion by applying functions on the method3 dataframe
#Important na functions we can use to do cleansing
strt11=StructType([StructField("id",StringType(),True),StructField("firstname",StringType(),True),StructField("lastname",StringType(),True),StructField("age",StringType(),True),StructField("profession",StringType(),True)])
dfmethod3=spark.read.schema(strt11).csv("/Volumes/we47catalog/we47schema/we47_volume/custsmodified",mode="PERMISSIVE",header=False)
display(dfmethod3.take(15))
print("Actual DF count",len(dfmethod3.collect()))
cleansed_df1=dfmethod3.na.drop(how="any")#drop the row, if any one column in our df row contains null
cleansed_df1=dfmethod3.na.drop(how="any",subset=["id","age"])#drop the row, if any one column id/age contains null
print("cleansed any DF count",len(cleansed_df1.collect()))
display(cleansed_df1.take(15))
cleansed_df2=dfmethod3.na.drop(how="all")#drop the row, if all the columns in our df row contains null
cleansed_df2=dfmethod3.na.drop(how="all",subset=["id","profession"])#drop the row, if all the columns (id,profession) in our df row contains null
print("cleansed all DF count",len(cleansed_df2.collect()))
display(cleansed_df2.take(15))

#Threshold - least bother (if we need minimum this many number of columns with not nulls, then only we will keep the row)
cleansed_df3=dfmethod3.na.drop(thresh=4,subset=["id","firstname","age","profession"])
print("cleansed threshold DF count",len(cleansed_df3.collect()))
display(cleansed_df3.take(15))

In [0]:
#Before scrubbing, lets take the right cleansed data with id as null and entire row as null removed out
#Finally I am arriving for our current data, lets perform the best cleansing
cleansed_df=cleansed_df3.na.drop(subset=["id"]).na.drop(how="all")
cleansed_df=cleansed_df.na.drop(subset=["firstname","lastname"],how='all')
print("Final cleansed DF",len(cleansed_df.collect()))
display(cleansed_df.take(15))

#####Scrubbing
It is a process of polishing/reformatting data or making the data more tidy Eg. polishing/scrubbing the mud portion of the potato

In [0]:
#Scrubbing will not remove the rows rather it will try to polish/scrub/reformat of the data in a usable format
#Can be achieved using 2 functions under na (na.fill & na.replace)
scrubbed_df1=cleansed_df.na.fill("na",subset=["firstname","lastname"]).na.fill("not provided",subset=["profession"])
scrubbed_df2=scrubbed_df1.na.replace("IT","Information Technologies",subset=["profession"]).na.replace("Pilot","Aircraft Pilot",subset=["profession"])
#or
#Find and Replace functionality
dict1={"IT":"Information Technologies","Pilot":"Aircraft Pilot","Actor":"Celebrity"}
scrubbed_df=scrubbed_df1.na.replace(dict1,subset=["profession"])
#scrubbed_df2=scrubbed_df1.na.replace("IT","Information Technologies",subset=["profession"]).na.replace("Pilot","Aircraft Pilot",subset=["profession"])
print("scrubbed DF",len(scrubbed_df.collect()))
display(scrubbed_df.take(15))


####3. Standardization, De-Duplication and Replacement / Deletion of Data to make it in a usable format

#####Standardization - 
Making the data more standard by adding/removing/reordering columns as per the expected standard, unifying into expected format, converting the type as expected etc.,

######Standardization1 - Column Enrichment (Addition of columns)

In [0]:
from pyspark.sql.functions import lit,initcap
#Standardization1 - Add columns (ENRICHMENT) to make this dataframe more standard for understanding of the source system
standard_df1=scrubbed_df.withColumn("sourcesystem",lit("Retail"))#If we have to add a columns with some hardcoded value in dataframe, we have use lit function to add a hardcoded/literal value
display(standard_df1.take(15))

######Standardization2 - Column Uniformity

In [0]:
#Standardization2 - UNIFORMITY of the data
display(standard_df1.groupBy("profession").count())#DSL
#standard_df1.createOrReplaceTempView("view1")
#display(spark.sql("select profession,count(1) from view1 group by profession"))#Declarative lang
standard_df2=standard_df1.withColumn("profession",initcap("profession"))#If we have to add a columns with some hardcoded value in dataframe, we have use lit function to add a hardcoded/literal value
display(standard_df2.take(15))
display(standard_df2.groupBy("profession").count())

######Standardization3 - Format Standardization

In [0]:
#Standardization3 - Format Standardization
from pyspark.sql.functions import *
cid_standardization={"one":"1","two":"2","ten":"10"}#We can think of using GenAI here later
standard_df3=standard_df2.na.replace(cid_standardization,subset=["id"])#Using munging feature for standardizing the data
standard_df3=standard_df3.withColumn("age",regexp_replace("age","-",""))
display(standard_df3.take(15))

######Standardization4 - Type Standardization

In [0]:
#I wanted to learn/test a function functionality
#create a dummy dataframe, apply function to that dummy df to test your function's functionality
spark.sql("select '123' as col1").where("col1 not rlike '[a-zA-Z]'").show()

In [0]:
standard_df3.printSchema()
#display(len(standard_df3.collect()))
#display(standard_df3.where("id like '%trailer%'"))
standard_df3=standard_df3.where("id not rlike '[a-zA-Z]'")#Removed the string data in the id column
display(standard_df3.where("id='trailer_data:end of file'"))
#display(len(standard_df3.collect()))
standard_df4=standard_df3.withColumn("age",col("age").cast("int")).withColumn("id",col("id").cast("long"))
standard_df4.printSchema()
display(standard_df4.take(15))
#standard_df4.where("id=1000").show()

######Standardization5 - Naming Standardization

In [0]:
standard_df5=standard_df4.withColumnRenamed("id","custid")
standard_df5.printSchema()
display(standard_df5.take(15))


######Standardization6 - Reorder Standadization

In [0]:
standard_df6=standard_df5.select("custid","age","firstname","lastname","profession","sourcesystem")
standard_df6.printSchema()
display(standard_df6.take(15))
#standard_df6.write.mode("overwrite").saveAsTable("munged_cust_data")

Capture all the functions or data munging functions onwards used in this entire notebook

#####DeDuplication - De-Duplication and removal of non prioritized dataset

In [0]:
#Duplicate Elimination at the record level, column level and in a priority basis of some column level higher age
dedup_df1=standard_df6.where("custid in (4000001,4000003)")
dedup_df1=standard_df6.distinct()#Eliminating Record level duplicates
dedup_df2=dedup_df1.dropDuplicates(subset=["custid"])#Retains only the first row and eliminate the subsequent rows with duplicate keys without having any other priority at the age or any other columns
#dedup_df1.where("custid in (4000001,4000003)").orderBy(["custid","age"],ascending=[True,False]).show()
#dedup_df2=dedup_df1.orderBy(subset=["custid","age"],ascending=[True,False]).coalesce(1)#Coalesing/combining all partition data into a single partition to get the dropduplicate output as expected
display(dedup_df2)
dedup_df3=dedup_df2.dropDuplicates(["custid"])#Retains only the first row and eliminate the subsequent rows with duplicate key
display(dedup_df3)
#dedup_df2.where("custid in (4000001,4000003)").show()
#How to cleanse the data with two same duplicate rows, except one of the column is different value?
#we have 5 columns, only profession column is varying?
#4000003	41	mohamed	irfan	Information Technologies N
#4000003 41	mohamed	irfan	Reporter Y
dedup_df4=dedup_df1.dropDuplicates(["custid","age","firstname","lastname"])

##**2. Data Enrichment** - Detailing of data
Makes your data rich and detailed <br>
a. Add (withColumn,select,selectExpr), Derive (withColumn,select,selectExpr), Remove/Eliminate (drop,select,selectExpr), Rename (withColumnRenamed,select,selectExpr), Modify/replace (withColumn, select/selectExpr) - (very important spark sql functions) <br>
b. split, merge/Concat <br>
c. Type Casting, reformat & Schema Migration <br>

![Stage2](stage2.png)

In [0]:
#Before we enrich, lets do some EDA
#Every stages we need to do basic EDA (Data Exploration)
dedup_df3.printSchema()
print("Records got cleaned/munged ",dfmethod1.count()-len(dedup_df3.collect()))
display(dedup_df3.summary())
display(dedup_df3)


#####a. Add (withColumn,select,selectExpr), Derive (withColumn,select,selectExpr), Rename (withColumnRenamed,select,selectExpr), Modify/replace (withColumn, select/selectExpr), Remove/Eliminate (drop,select,selectExpr) - (very important spark sql functions)

######Adding of columns

In [0]:
#Adding columns
datadt="25-12-25"#source system data generated date
enrich_df1=dedup_df3.withColumn("loaddt",current_date()).withColumn("datadt",lit(datadt))
enrich_df1.show(2)
enrich_df1=dedup_df3.withColumns({"loaddt":current_date(),"datadt":lit(datadt)})
enrich_df1.show(2)
#or
enrich_df1=dedup_df3.withColumn("sourcesystem1",col("sourcesystem"))#Adding an existing column in a new column (copying a column value to a new column)
enrich_df1.show(2)
#or
enrich_df1=dedup_df3.select("*",current_date().alias("loaddt"),lit(datadt).alias("datadt"))#pure DSL
enrich_df1.show(2)
#or
#colvar=f"{datadt} as datadt"
datadt="25-12-25"
enrich_df1=dedup_df3.selectExpr("*","current_date() as loaddt",f"'{datadt}' as datadt")#Only select Function that supports both DSL and SQL
enrich_df1.show(2)


######Deriving of columns

In [0]:
enrich_df2=enrich_df1.withColumn("profession_flag",substring("profession",1,1))
enrich_df2.show(10)
#or we can achieve using select
enrich_df2=enrich_df1.select("*",substring("profession",1,1).alias("profession_flag"))
enrich_df2.show(10)
#or we can achieve using selectExpr
enrich_df2=enrich_df1.selectExpr("*","substr(profession,1,1) as profession_flag")
enrich_df2.show(10)

######Renaming of columns

In [0]:
enrich_df3=enrich_df2.withColumnRenamed("profession_flag","proflag")#better to use
enrich_df3.show(10)
enrich_df3=enrich_df2.withColumnsRenamed({"profession_flag":"proflag","profession":"prof"})
enrich_df3.show(10)
#or
enrich_df3=enrich_df2.select("*",col("profession").alias("prof"))#This will derive a new column called prof
enrich_df3=enrich_df2.select("custid","age","firstname","lastname",col("profession").alias("prof"),"sourcesystem","loaddt","datadt",col("profession_flag").alias("proflag"))#This will derive a new column called prof#better to use
#enrich_df3=enrich_df3.drop("profession")#column orders are changed, not good to use
enrich_df3.show(10)

######Modify/replace (withColumn, select/selectExpr)

In [0]:
enrich_df3.printSchema()#datadt is not in a expected date format for further usage and i wanted to convert sourcesystem into uppercase
enrich_df4=enrich_df3.withColumn("sourcesystem",upper(col("sourcesystem"))).withColumn("datadt",to_date(col("datadt"),'dd-MM-yy'))#This time withColumn didnt added a new column, it just modified the existing column
enrich_df4.printSchema()#datadt is expected date format
enrich_df4.show(10)

######Remove/Eliminate (drop,select,selectExpr) 

In [0]:
enrich_df5=enrich_df4.withColumn("fullname",concat(col("firstname"),lit(" "),col("lastname")))
enrich_df5=enrich_df5.drop("firstname","lastname").select("custid","age","fullname","prof","sourcesystem","loaddt","datadt","proflag")#drop function is good to use if we just want to drop a column in the df
display(enrich_df5.limit(10))
#or
enrich_df5=enrich_df4.select("custid","age",concat(col("firstname"),lit(" "),col("lastname")).alias("fullname"),"prof","sourcesystem","loaddt","datadt","proflag")#Better methodology
display(enrich_df5.limit(10))

######Conclusion/Best practices of using different column enrichment functions
1. select is good to use if we want to perform - 
Good for ordering/reordering, only renaming column (not good), only reformatting/deriving a column (not good), **for all of these operation in a single iteration** such renaming, reordering, reformatting,deriving, dropping etc., (best to use)
2. selectExpr is good to use if we want to perform - Same as select by using iso sql functionality (if we are not familiar in DSL) **for all of these operation in a single iteration**
3. withColumn is good to use if we want to perform - 
**for adding/deriving/modifying/replacing in a single iteration**
Adding/Deriving column(s) in the last (Good), Modifying/replacing (Good), Renaming (not good), Dropping(not possible)
4. withColumnRenamed is good to use if we want to perform - only for renaming column (Good)
5. drop is good to use if we want to perform - only dropping of columns in the given position (Good)

#####b. Splitting & Merging/Melting of Columns

In [0]:
#Splitting of columns
enrich_df6=enrich_df5.withColumn("profsplit",split(col("prof"),' '))
enrich_df6=enrich_df6.withColumns({"proffirst":col("profsplit")[0],"proflast":col("profsplit")[size(col("profsplit"))-1]})
display(enrich_df6.limit(10))

In [0]:
#Merging of columns
enrich_df7=enrich_df6.withColumn("proflag",concat(substring(col("proffirst"),1,1),substring(col("proflast"),1,1))).drop("profsplit")
display(enrich_df7)

#####c. Typecasting & Formatting

In [0]:
#Consider the below casting and formatting
enrich_df7.printSchema()#datadt is not in a expected date format for further usage and i wanted to convert sourcesystem into uppercase
#enrich_df4=enrich_df3.withColumn("sourcesystem",upper(col("sourcesystem"))).withColumn("datadt",to_date(col("datadt"),'dd-MM-yy'))#This time withColumn didnt added a new column, it just modified the existing column
#enrich_df4.printSchema()#datadt is expected date format
#enrich_df4.show(10)

##3. Data Customization & Processing - Application of Tailored Business specific Rules <br>
a. User Defined Functions <br>
b. Building of Frameworks & Reusable Functions

![Stage3](stage3.png)

######1. Example udf

In [0]:
#1. Import udf library
from pyspark.sql.functions import udf
#2. Create a Python function - Can only work in single local computer
def convert_to_upper(name):#python function can only run in irfan's computer
    return name.upper()
#3. Register the python function as user defined function, hence it supports distributed processing of data on dataframe columns
user_def_fun=udf(convert_to_upper)
#4. Apply the udf in my dataframe columns - enrich_df7(10k) -> nandha 5k, senthil 5k
enrich_df8=enrich_df7.withColumn("fullname",user_def_fun(col("fullname")))
display(enrich_df8.limit(10))

######2. Interview Question: Have you created any custom/udfs in your project, if so what you did out of that, is it good to use?
1. Yes I created for some custom/business specific requirement, which has some complex logics to achive which is not achievable in a regular sql query. for eg. taxation business rule is very specific to my organization/state.
or
No, We are not suppose to create udfs as it affects the performance of the pipeline, hence we tried implementing most of business logics in our sql itself/ We created very limited udfs considering performance bottleneck only if it is inevitable.

Interview Question: Why UDFs are creating performance bottleneck?
Ans: UDFs are blackbox for the Spark optimizer engine (Catalyst/Photon) can't understand the function that we created. Refer the code below.

is there any exceptional mechanism in spark to resolve the performance issue if we go with custom function ? If we use udf, there is no option to fix this performance issue, try to convert udf to sql functionality...

In [0]:
print("Builtin function")
rawdf1.withColumn("upperfname",upper("firstname")).explain()
#== Photon Explanation ==
#The query is fully supported by Photon/Catalyst.

print("Custom UDF function")
rawdf1.withColumn("upperfname",user_def_fun("firstname")).explain()
#Photon/Catalyst does not fully support the query because:
#Unsupported node: BatchEvalPython [convert_to_upper(firstname#19759)#19778], [pythonUDF0#19781].


######3. Agecategory User defined function

In [0]:
#Lets Create an UDF to calculate agecateogy of customers
#1. Import udf library
from pyspark.sql.functions import udf
#2. Create a Python function - Can only work in single local computer
def agecat(agecol):
    if agecol<11:
        return "Children"
    elif agecol>=11 and agecol<=19:
        return "Teens"
    elif agecol>=20 and agecol<=50:
        return "Middleage"
    else:
        return "Oldage"
    
#3. Register the python function as user defined function, hence it supports distributed processing of data on dataframe columns
agecatudf=udf(agecat)#This python function is now a spark ready distributed function
#4. Apply the udf in my dataframe columns - enrich_df7(10k) -> nandha 5k, senthil 5k
custom_df=enrich_df7.withColumn("agecat",agecatudf(col("age")))
display(custom_df.limit(20))

##4. Data core Curation/Processing (Pre Wrangling) - Applying different levels of business logics, transformation, filtering, grouping, aggregation, sorting and limits applying different transformation functions
1. Select, Filter
2. Derive flags & Columns
3. Format
4. Group & Aggregate
5. Sorting
6. Limit

![Curation](stage4.png)

#####1.Select, Filter

In [0]:
#Select - column selection, reordering, column level filtering
selectdf=custom_df.select("custid","age",upper("prof").alias("profession"))#DSL
selectdf.show(1)
selectdf=custom_df.selectExpr("custid","age","upper(prof) as profession")#SQL
selectdf.show(1)
#Where/Filter -We did row level filtering applying some conditions (SQL/DSL fashion)
filterdf=selectdf.where((col("age")>30) | (col("age")<60))#DSL Syntax
filterdf.show(1)
filterdf=selectdf.where("age>30 or age<60")#SQL Syntax
filterdf.show(1)

#where or filter, which one to use? Both are aliases (filter is used by fbp people and where is used by sql people)
#display(filterdf.limit(20))

In [0]:
#Question : Whether we can apply select -> where or where -> select
#Answer : Both are same, spark (photon/catalyst) will do row level filter first
selectfilterdf=rawdf1.selectExpr("id","age","upper(profession) as profession").where("age>30 or age<60")
selectfilterdf.show(1)
selectfilterdf.explain()
#or 
selectfilterdf=rawdf1.where("age>30 or age<60").selectExpr("id","age","upper(profession) as profession")
selectfilterdf.show(1)
selectfilterdf.explain()

#####2. Derive flags & Columns
We are achieving by using case statements

In [0]:
#Deriving Columns (Age category)
#Is there any exceptional mechanism in spark to resolve the performance issue if we go with custom function ? If we use udf, there is no option to fix this performance issue, try to convert udf to sql functionality...
#We will try to achieve the same functionality of the udf by using sql functions (better option)
'''def agecat(agecol):
    if agecol<11:#when(condition,return something)
        return "Children"
    elif agecol>=11 and agecol<=19:
        return "Teens"
    elif agecol>=20 and agecol<=50:
        return "Middleage"
    else:
        return "Oldage"
'''
enrich_df7.createOrReplaceTempView("enrichview")
#SQL Syntax: case when condt then expression when cond2 then expression2 else expression end
#spark.sql("select *,case when age<11 then 'Children' when age>=11 and age<20 then 'Teens' when age>=20 and age<=50 then 'Middleage' else 'Oldage' end as agecat from enrichview").show(2)
spark.sql("""select *,
          case when proffirst=proflast then upper(substr(proffirst,1,2)) 
          else upper(concat(substring(proffirst,1,1),substr(proflast,1,1)))
          end as profind 
          from enrichview""").show(2)

#DSL Syntax: when(condition,return something).when((cond1)&(cond2),return something).otherwise(return something else)
derived_df1=enrich_df7.withColumn("agecat",
                                 when((col("age")<11),lit("Children")).
                                 when(((col("age")>=11) & (col("age")<20)),lit("Teens")).
                                 when(((col("age")>=20) & (col("age")<=50)),lit("Middleage")).
                                 otherwise(lit("Oldage")))
#DSL Syntax to derive indicator for profession
derived_df2=derived_df1.withColumn("profind",when(((col("proffirst"))==(col("proflast"))),upper(substring(col("proffirst"),1,2))).otherwise(upper(concat(substring("proffirst",1,1),substring(col("proflast"),1,1)))))
derived_df2.show(20)

#####3.Format (deriving columns)

In [0]:
formatted_derived_df=derived_df2.select("*",date_format(col("datadt"),'yyyyMM').alias('datadtyrmth')
                                         ,year("datadt").alias("datadtyr")
                                         ,month("datadt").alias("datadtmth")
                                         ,date_diff("loaddt","datadt").alias("delayindays")
                                         ,last_day("datadt").alias("datadtlastday")
                                         ,date_add("loaddt",1).alias("nextday"))
formatted_derived_df.show(2)

#####4. Group & Aggregate
Mostly used in data analytics, but Transformation also we will do<br>
Thumb rules:<br>
1. group by - dimension of the data, column should be low in cardinality/high frequency, it can be descriptive/flag/indicators<br>
2. aggregation columns - measures of the data, It can high cardinal, mostly number/date data types, measurable value

In [0]:

grouped_df=formatted_derived_df.groupBy("profind")#It will just group
display(grouped_df)
#Calculate profession and agecategory wise count of customers
grouped_agg_df=formatted_derived_df.groupBy("prof","agecat").count()
grouped_agg_df.show(2)
#Not preferred (only for single column) Calculate profession and agecategory wise avg age of customers
grouped_agg_df=formatted_derived_df.groupBy("prof").avg("age").withColumnRenamed("avg(age)","avgage")
grouped_agg_df.show(2)
#More preferred to use agg(aggregate functions) Calculate profession and agecategory wise avg age of customers
grouped_agg_df=formatted_derived_df.groupBy("prof").agg(avg("age").alias("avgage"),min("age").alias("minage"),max("age").alias("maxage"))
grouped_agg_df.show(10)
grouped_agg_df=formatted_derived_df.groupBy("prof","agecat").agg(avg("age").alias("avgage"),min("age").alias("minage"),max("age").alias("maxage"))
grouped_agg_df.show(10)#We will use this grouped aggregated data later post wrangling operation (last stage)

#####5. Ordering

In [0]:
grouped_agg_df.orderBy("prof","agecat").show(20)#by default ascending order of sorting
grouped_agg_df.orderBy(["prof","agecat"],ascending=[True,False]).show(20)#by default ascending,descending order of sorting
#We will use this grouped aggregated sorted data later post wrangling operation (last stage)

#####6. Limit
Difference between limit, take and show/display:
1. limit(transformation) is used to limit the number of rows in the dataframe (can be placed anywhere)
2. take (action) is used to limit the number of rows in the dataframe and return the result as a python list of rows (can be placed in the last)
3. show is used to display the dataframe in a tabular format
4. display (databricks specific) is used to display the dataframe in a formatted/advanced tabular format where we can download/visualize etc.,

In [0]:
grouped_agg_df.orderBy(["prof","agecat"],ascending=[True,False]).limit(10).show(20)
grouped_agg_df.orderBy(["prof","agecat"],ascending=[True,False]).take(10)
display(grouped_agg_df.orderBy(["prof","agecat"],ascending=[True,False]).take(10))

##5. Data Wrangling - Transformation & Analytics
  1. Joins<br>
    1. Frequently used simple joins (inner, left)<br>
    2. InFrequent simple joins (self, right, full, cartesian)<br>
    3. Advanced joins (Semi and Anti)<br>
    4. Join Optimazation in Distributed platform(Broadcast join, SMB join, M/R side join, Shuffle Hash join, skewed join(avoided))
  2. Lookup
  3. Lookup & Enrichment
  4. Schema Modeling  (Denormalization)
  5. Windowing
  6. Analytical
  7. Set operations
  8. grouping & aggregations (advanced)

![Stage5](stage5.png)

#####1. Joins
Joins are Relation/connection of one or more tables to perform widened (horizontal) data analytics
1. Frequently used simple joins (inner, left)
2. InFrequent simple joins (self, right, full, cartesian)
3. Advanced joins (Semi and Anti)
or<br>
Additional surrounding concepts of Joins: Relationships(1:1/1:many/many:many), schema modeling(star/snowflakes) Normalization/denormalization,  <br>
Categories of Joins:<br>
1.Inner join<br>
Type: inner/equi/natural join (important)<br>

2.Outer join<br>
a.Left (important)<br>
b.Right (We can achieve using left outer by swapping tables)<br>
c.Full<br>

3.Non Equi Join<br>
Type: cross/cartesian (should be avoided)<br>

4. Self join (we can achieve using inner join and we do Related analytics with the eligible data joining the same table)<br>
eg. <br>
custid,name,refcustid<br>
1,irfan,1<br>
2,bhargavi,1<br>
3,nancy,1<br>
4,bharathi,3<br>

5. Specialized left join<br>
a. semi<br>
b. anti<br>

6. Optimized Join (This we learn later post architecture revisit)<br>
a. broadcast/map side join<br>
b. Sort Merge bucket join / reduce side join<br>
c. Shuffle hash join / reduce side join<br>
d. skewed join



In [0]:
custdf1=formatted_derived_df
df1=custdf1.where("custid in (4000100,4000101,4000108)")
df2=custdf1.where("custid in (4000100,4000102,4000103)")
df1=df1.withColumn("custid",when((col("custid")==4000108),lit(None)).otherwise(col("custid")))
df2=custdf1.where("custid in (4000100,4000102,4000103,4000104)")
df2=df2.withColumn("custid",when((col("custid")==4000104),lit(None)).otherwise(col("custid")))
display(df1)
display(df2)
print("inner")
display(df1.join(df2,how='inner',on='custid'))#1
print("semi")
display(df1.join(df2,how='semi',on='custid'))#1
print("left")
display(df1.join(df2,how='left',on='custid'))#2
print("anti")
display(df1.join(df2,how='anti',on='custid'))#2
print("right")
display(df1.join(df2,how='right',on='custid'))#3
print("full")
display(df1.join(df2,how='full',on='custid'))#4


In [0]:
from pyspark.sql.functions import *
#Requirement: I need to do analytics/reporting of top 3 customers who did highest amount of transactions in our business last year, so i can send them some offers.
#Solution: First Lets try to bring transaction dataset to perform joins
strt1="txnid long,txndt string,custid long,amt float,cat string,prod string,city string,state string,spendby string"
txnsdf=spark.read.schema(strt1).csv("/Volumes/workspace/default/volumewe47_datalake/txns_2025.txt",header=False)
#print(len(txnsdf.collect()))
txnsdf_munged=txnsdf.distinct().dropDuplicates(["txnid"]).na.drop()
#print(len(txnsdf_munged.collect()))
txnsdf_munged.printSchema()
txnsdf2=txnsdf_munged.withColumn("txndt",to_date("txndt",'MM-dd-yyyy'))
txnsdf2.printSchema()
display(txnsdf2.summary())
display(txnsdf2.take(20))

In [0]:
#Requirement: I need to do analytics/reporting of top 3 customers who did more and highest sum amount of transactions in our business last year, so i can send them some offers.
#First - Let us perform join to achieve the expected requirement result (one to many join)
#Second - Do grouping & aggregation
#Third - Sorting
#Fourth - limiting
custdf1=formatted_derived_df
#custdf1.printSchema()
joineddf=custdf1.join(other=txnsdf2,on="custid",how="inner")#Preferable join is "inner" (whether you mention or not)
print(joineddf.count())
joineddf.groupBy("custid","fullname").count().orderBy(desc("count")).show(3)#Top 3 customers visited our shop mostly
top3=joineddf.groupBy("custid","fullname","prof").agg(sum("amt").alias("tot_amt"))\
.withColumn("custtype",lit("top3"))\
.orderBy("tot_amt",ascending=False)\
.limit(3) #Top 3 customers purchased for highest amount last year
least3=joineddf.groupBy("custid","fullname","prof").agg(sum("amt").alias("tot_amt")).withColumn("custtype",lit("least3")).orderBy("tot_amt",ascending=True).limit(3)#Top 3 customers purchased for lowest amount last year
display(top3.union(least3))

#or (For performance optimization interview answer)
#First - Do grouping & aggregation
grouped_cust_txns=txnsdf2.groupBy("custid").agg(sum("amt").alias("tot_amt"))
#Second - Let us perform join to achieve the expected requirement result (one to one join)
joineddf=custdf1.join(other=grouped_cust_txns,on="custid",how="inner")
#Third - Sorting
top3=joineddf.select("custid","fullname","prof","tot_amt").withColumn("custtype",lit("top3"))\
.orderBy("tot_amt",ascending=False)
#Fourth - limiting
top3=top3.limit(3)
display(top3)

In [0]:
#Requirement: I need to do analytics/reporting of the dormant/in active customer who didn't visited our business last year
#custdf1 dataframe contains all customer info of the complete years of my business
#txnsdf2 dataframe contains last 1 year transactions
custdidnotvisitedlastyeardf=custdf1.join(txnsdf2,on="custid",how="left")#Returns all data from left df and only matching data from right df rest of data will be having nulls
display(custdidnotvisitedlastyeardf.where("txnid is null"))#inactive customers

In [0]:
#Requirement: I need to do analytics/reporting of the total number of active customer who visited our business last year
#custdf1 dataframe contains all customer info of the complete years of my business
#txnsdf2 dataframe contains last 1 year transactions
custdidnotvisitedlastyeardf=custdf1.join(txnsdf2,on="custid",how="left")#Returns all data from left df and only matching data from right df rest of data will be having nulls
print(custdidnotvisitedlastyeardf.where("txnid is not null").selectExpr("custid").distinct().count())#active customers of last 1 year

#how many active customers of 2025 december
txnsdfdec2025=txnsdf2.where("month(txndt)=12")
custdidnotvisiteddecmonthdf=custdf1.join(txnsdfdec2025,on="custid",how="left")
print("inactive customers of december month",custdidnotvisiteddecmonthdf.where("txnid is null").selectExpr("custid").distinct().count())
print("active customers of december month",custdidnotvisiteddecmonthdf.where("txnid is not null").selectExpr("custid").distinct().count())

#I want achieve right outer join without using right keyword?
custdidnotvisitedlastyeardf=txnsdf2.join(custdf1,on="custid",how="left")#this is right join now

In [0]:
#how many products are transacted in december month regardless of customer information
custdidnotvisiteddecmonthdf=custdf1.join(txnsdfdec2025,on="custid",how="right")
display(custdidnotvisiteddecmonthdf)
print("Overall number of products ",txnsdfdec2025.select("prod").distinct().count())
print("Products had no purchases by the inactive customers",custdidnotvisiteddecmonthdf.where("age is null").selectExpr("prod").distinct().count())#active products of december month


In [0]:
#how many products are transacted in december month (show me both customer and products regardless of customer information or product information matching)
custdidnotvisiteddecmonthdf=custdf1.join(txnsdf2,on="custid",how="full")
print(custdidnotvisiteddecmonthdf.count())
#print("Overall number of products ",txnsdfdec2025.select("prod").distinct().count())
print("Total Products available ",custdidnotvisiteddecmonthdf.selectExpr("prod").distinct().count())#active products of december month
print("Products had purchases by customers",custdidnotvisiteddecmonthdf.where("age is not null").selectExpr("prod").distinct().count())#active products of december month
print("Products not purchased by customers",custdidnotvisiteddecmonthdf.where("age is null").selectExpr("prod").distinct().count())#active products of december month
print("Total customer available, but no transaction made ",custdidnotvisiteddecmonthdf.where("txnid is null").selectExpr("fullname").distinct().count())#active products of december month
print("Total customer available, transaction made ",custdidnotvisiteddecmonthdf.where("txnid is not null").selectExpr("fullname").distinct().count())#active products of december month



######1. Lookup 
semi/anti (best), left/inner (least bother)
Lookup and enrichment is the process of looking up for some data attributes using the key to identify the presence of the values (not the actual values are returned)
lookup? existence check that returns boolean
Eg. whether this particular customer made a transaction or not


In [0]:
#I need to produce only the df1 data present in df2 (by doing lookup)
df1=custdf1.where("custid in (4000100,4000101)")
df1.show()
df2=custdf1.where("custid in (4000100,4000102,4000103)")
#With the left customer info provided, check whether it is present in the right df and display both left and right matching data
display(df1.alias("a").join(df2.alias("b"),how='inner',on='custid').select(col("a.*")))#Returns only the matching data from both left & right df also, comparing with right, hence Inner join will lookup and enrich also

#With the left customer info provided, check whether it is present in the right df and display only left matching data (right)
display(df1.join(df2,how='left_semi',on='custid'))#Returns only the matching data from the left df, comparing with right - hence semi is a lookup join

In [0]:
#Requirement: I need to do analytics/reporting of the total number of active customer who visited our business last year
#custdf1 dataframe contains all customer info of the complete years of my business
#txnsdf2 dataframe contains last 1 year transactions
custdidnotvisitedlastyeardf=custdf1.join(txnsdf2,on="custid",how="left_semi")#Returns all data from left df only that matches with the right df
print("Total active customers of last year ",custdidnotvisitedlastyeardf.count())
#display(custdidnotvisitedlastyeardf.where("txnid is not null").selectExpr("custid").distinct().count())#inactive customers of last 1 year

In [0]:
#Requirement: I need to do analytics/reporting of the total number of IN active customer who didn't visited our business last year
custdidnotvisitedlastyeardf=custdf1.join(txnsdf2,on="custid",how="leftanti")#Returns all data from left df only that DID NOT matches with the right df
display(custdidnotvisitedlastyeardf)
print("Total IN active customers of last year ",custdidnotvisitedlastyeardf.count())

######2. Lookup & Enrichment
Lookup and enrichment using joins - left_join (best), inner, right(least bother)
Lookup and enrichment is the process of looking up for for some data attributes using the key and enrich the values.
Eg. Checking the bank account balance for a given acct id.
lookup & enrichment? existence check that returns result/value that can be used for enriching our data

In [0]:

lookup_enriched_df=custdf1.join(txnsdf2,how='left',on='custid')
display(lookup_enriched_df.where("txnid is not null"))
display(lookup_enriched_df.where("txnid is null"))

In [0]:
lookup_enriched_df=custdf1.join(txnsdf2,how='inner',on='custid')
#display(lookup_enriched_df.where("txnid is not null"))#Only the matching data between left and right alone returned
#display(lookup_enriched_df.where("txnid is null"))#no data returned
display(lookup_enriched_df)

######3.Schema Modeling (Denormalization)
Use inner(mostly),left/right/full(we can use depends on the business)
We can build Datawarehouse components (dimension, fact tables) appling joins on the tables to achieve different types of schemas
1. Star Schema
2. Snowflake Schema
3. Starflake Schema

In [0]:
#Implementing only star schema model (fact surrounded by one or more main dimension) to produce denormalized (single dataset) fact table
dimensiondf=custdf1
#display(dimensiondf)
factdf=txnsdf2
#display(factdf)
final_denormalized_df1=dimensiondf.join(factdf,how='inner',on='custid')#applying inner to reject/miss data from both side
final_denormalized_df2=dimensiondf.join(factdf,how='left',on='custid')#applying left outer to ensure all customer data is present
final_denormalized_df3=dimensiondf.join(factdf,how='right',on='custid')#applying right outer to ensure all transaction data is present
final_denormalized_df4=dimensiondf.join(factdf,how='full',on='custid')#applying full outer to not reject/miss any data
display(final_denormalized_df1)#Final fact table to load in our lakehouse(delta table)

#####5.Windowing
row_number, rank, dense_rank<br>
Application of Windowing & Analytical functions<br>
Most frequent interview questions asked...<br>
1. how do you create surrogate/seq number in tables?<br>
2. Top N analysis of data?<br>
3. how do you remove duplicates in the table data?<br>
select * from (select *,row_number() over(partition by department_id order by salary) rownum
from employees)temp
where rownum=1;

In [0]:
#Creation of Surrogate/Sequence Numbers
#1. how do you create surrogate/seq number in tables? 
# If the source given some key (natural key) and we wanted to maintain our own key in our environment then we create surrogate key (unique key with sequence) Eg. For a bank - Aadharnumber(naturalkey) & bankacctnumber(surrogatekey)
from pyspark.sql.window import Window
dfcusts=final_denormalized_df1
dfcusts_surrogate_seqnum_df=dfcusts.select(row_number().over(Window.orderBy("custid","amt")).alias("primary_sk"),"*")
#Performance degradation: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
#or 
#One more efficient way to create surrogate key without sorting the data (Interview question)
dfcusts_surrogate_seqnum_df=dfcusts.select(monotonically_increasing_id().alias("primary_sk"),"*")
display(dfcusts_surrogate_seqnum_df)

In [0]:
#2. Top N Analysis (Very very important for interview..)
from pyspark.sql.window import Window
dfcusts=final_denormalized_df1.select("custid","fullname","prof","txnid","amt","prof").where("custid in (4000001,4000002,4000003)")
#display(dfcusts)

print("1.Row number")
custtrans_topn_df=dfcusts.withColumn("custrno",row_number().over(Window.partitionBy("custid").orderBy(desc("amt"))))
display(custtrans_topn_df)
custtrans_topn_df.where("custrno=1").show()#Show me the only top 1 transaction made by the every customer
custtrans_topn_df.where("custrno=2").show()#Second Highest transaction made by each customer
custtrans_topn_df.where("custrno=5").show()#Fifth Highest transaction made by each customer (row_number will return only 1 row)
custtrans_topn_df.where("custrno<=3").show()#Show me the top 3 transactions made by the customers (with the same values of transaction)

custtrans_topn_df=dfcusts.withColumn("custrno",row_number().over(Window.partitionBy("custid").orderBy(asc("amt"))))
display(custtrans_topn_df)
custtrans_topn_df.where("custrno=1").show()#Least transaction made by each customer
custtrans_topn_df.where("custrno=2").show()#Second Least transaction made by each customer

print("Rank & Dense_rank usage:")
custtrans_topn_df=dfcusts.withColumn("custrnk",rank().over(Window.partitionBy("custid").orderBy(asc("amt")))).withColumn("custdrnk",dense_rank().over(Window.partitionBy("custid").orderBy(asc("amt"))))
display(custtrans_topn_df)
print("Rank")
#Rank returns multiple rows per customer if the top transaction is same value
custtrans_topn_df.where("custrnk=1").show()#Show me the only top 1 transaction made by the every customer 
custtrans_topn_df.where("custrnk=2").show()#Second Highest transaction made by each customer
custtrans_topn_df.where("custrnk=4").show()#Four Highest transaction made by each customer (row_number will return only 1 row)
custtrans_topn_df.where("custrnk=5").show()#this record is missing (because skipped by rank function)
custtrans_topn_df.where("custrnk=6").show()#this record is missing (because skipped by rank function)

print("Dense Rank")
#Dense Rank returns multiple rows per customer if the top transaction is same value and the next value of the rank is dense/not skipped
custtrans_topn_df.where("custdrnk=1").show()#Show me the only top 1 transaction made by the every customer 
custtrans_topn_df.where("custdrnk=2").show()#Second Highest transaction made by each customer
custtrans_topn_df.where("custdrnk=4").show()#Fifth Highest transaction made by each customer (row_number will return only 1 row)
custtrans_topn_df.where("custdrnk=5").show()#this record is available (because dense rank function is used)
custtrans_topn_df.where("custdrnk=6").show()#this record is available (because dense rank function is used)


print("Rownumber, Rank & Dense_rank usage:")
custtrans_topn_df=dfcusts.withColumn("custrno",row_number().over(Window.partitionBy("custid").orderBy(asc("amt")))).withColumn("custrnk",rank().over(Window.partitionBy("custid").orderBy(asc("amt")))).withColumn("custdrnk",dense_rank().over(Window.partitionBy("custid").orderBy(asc("amt"))))
display(custtrans_topn_df)

#####6.Analytical
Functions used for performing some analytical operations on the data in multiple dimensions. We can use windowing or grouping or grouping & aggregation functions to perform analytics/analytical functionalities
1. Hierarchical Analytics (lead & lag)
2. Aggregation analytics (grouping & aggregations)
3. Multi dimensional analytics (rollup, cube, pivot)


In [0]:
#Let us perform hierarchical analytics by using lead and lag functions
from pyspark.sql.window import Window
dfcusts=final_denormalized_df1.select("custid","fullname","prof","txnid","amt","prof","txndt").where("custid in (4000001,4000002,4000003)")
#Lead help us arrive the next value
hierarchicalleaddf=dfcusts.withColumn("nextamt",lead("amt").over(Window.partitionBy("custid").orderBy("txndt")))
display(hierarchicalleaddf)
#Lag help us arrive the prior value
hierarchicallagdf=dfcusts.withColumn("prioramt",lag("amt").over(Window.partitionBy("custid").orderBy("txndt")))
display(hierarchicallagdf)

print("application of lead and lag")
print("How to control the hierarchy (for example I want to compare immediate next transaction or 2 levels of transactions")
#syntax lead(col,offset,default) or lag(col,offset,default)
hierarchicalleaddf=dfcusts.withColumn("nextamt",lead("amt",2,-1).over(Window.partitionBy("custid").orderBy("txndt")))
display(hierarchicalleaddf)
#Lead help us arrive the prior value
hierarchicallagdf=dfcusts.withColumn("prioramt",lag("amt",2,0).over(Window.partitionBy("custid").orderBy("txndt")))
display(hierarchicallagdf)

print("Requirement: I want to analyse the purchase pattern/trend of my customer")
hierarchicalleadlagdf=dfcusts.withColumn("nextamt",lead("amt",1,0).over(Window.partitionBy("custid").orderBy("txndt")))\
.withColumn("prioramt",lag("amt",1,0).over(Window.partitionBy("custid").orderBy("txndt")))
display(hierarchicalleadlagdf)
trendanalysisdf=hierarchicalleadlagdf.withColumn("trend",when((col("nextamt")>col("amt")),"up").when((col("nextamt")<col("amt")),"down").otherwise("same"))
display(trendanalysisdf)

2. Aggregation analytics (grouping & aggregations)

In [0]:
dfcusts=final_denormalized_df1.select("custid","fullname","txnid","amt","prof","txndt","age","state","city","spendby").where("custid between 4000001 and 4000500")
dfcusts.show(2)
groupedaggdf=dfcusts.groupBy("prof").agg(sum("amt").alias("sumamt"),avg("age").alias("avgage"))
display(groupedaggdf)
groupedaggdf=dfcusts.groupBy("state","city","prof").agg(sum("amt").alias("sumamt"),avg("age").alias("avgage"))
display(groupedaggdf)

In [0]:
#Multi dimensional analytics (rollup, cube, pivot)
print("rollup produces the aggregated result in rolled UP dimensions")
groupedaggdf=dfcusts.rollup("state","prof").agg(sum("amt").alias("sumamt"),avg("age").alias("avgage")).orderBy("state","prof")
display(groupedaggdf)

print("cube produces the aggregated result in all dimensions (rollup and rolldown)")
groupedaggdf=dfcusts.cube("state","prof").agg(sum("amt").alias("sumamt"),avg("age").alias("avgage")).orderBy("state","prof")
display(groupedaggdf)

In [0]:
#Pivot operation (costly function)
print("pivot produces the aggregated result of the dimensions in a columnar fashion")
groupedaggdf=dfcusts.groupBy("state","city","spendby").agg(sum("amt").alias("sumamt"),avg("age").alias("avgage")).orderBy("state","city","spendby")
display(groupedaggdf)
groupedaggdf=dfcusts.groupBy("state","city").pivot("spendby").agg(sum("amt").alias("sumamt"),avg("age").alias("avgage")).orderBy("state","city")
display(groupedaggdf)

#####7.Set operations

In [0]:
#Set operation (union, unionAll, insetsect, minus/except/subract)
dfcusts1=final_denormalized_df1.select("custid","fullname","txnid","amt","prof","txndt","age","state","city","spendby").where("custid between 4000001 and 4000010")
dfcusts2=final_denormalized_df1.select("custid","fullname","txnid","amt","prof","txndt","age","state","city","spendby").where("custid between 4000005 and 4000015")
unionalldf=dfcusts1.unionAll(dfcusts2)
#display(unionalldf)#returns all data from both the dataframes with duplicates
print(unionalldf.count())
uniondf=dfcusts1.union(dfcusts2).distinct()
display(uniondf)#returns all data from both the dataframes with duplicates  (it will not work like this in spark), we have to use distinct explicitly
print(uniondf.count())

#intersection
commondf=dfcusts1.intersect(dfcusts2)
display(commondf)
print(commondf.count())

#subtract
diffdf=dfcusts1.subtract(dfcusts2)#returns df1-df2 data (excessive df1 data)
display(diffdf)
print(diffdf.count())


##6. Egress of the required DFs created at any of the above stages, we have persist in the tables or in a form of files or to some external systems.

1. how to create pipelines using different data processing techniques by connecting with different sources/targets
2. how to Standardize/Modernization/Industrializing the code and how create/consume generic/reusable functions & frameworks
3. Testing (Unit, Peer Review, SIT/Integration, Regression, User Acceptance Testing), Masking engine,
4. Reusable transformation(munge_data, optimize_performance),
5. Quality suite/Data Profiling/Audit engine (Reconcilation) (Audit framework), Data/process Observability

6. how terminologies/architecture/submit jobs/monitor/log analysis/packaging and deployment ...
7. performance tuning
8. Deploying spark applications in Cloud & other Distributions like Hortonworks/Cloudera/Databricks
9. Creating cloud pipelines using spark SQL programs & Cloud native tools

What is the importance of learning this program or How this can address interview questions..?
VERY VERY IMPORTANT PROGRAM IN TERMS OF EXPLAINING/SOLVING PROBLEMS GIVEN IN INTERVIEW ,
WITH THIS ONE PROGRAM YOU CAN COVER ALMOST ALL DATAENGINEERING FEATURES
Tell me about the common transformations you performed,
tell me your daily roles in DE,
tell me some business logics you have writtened recently
How do you write an entire spark application,
levels/stages of DE pipelines or
have you created DE pipelines what are the transformations applied,
how many you have created or are you using existing framework or you created some framework?

'''
TRANSFORMATION & ANALYTICAL TECHNIQUES
Starting point - (Data Governance (security) - Tagging, categorization, classification, masking/filteration)
1. Data Munging - Process of transforming and mapping data from Raw form into Tidy(usable) format with the
intent of making it more appropriate and valuable for a variety of downstream purposes such for
further Transformation/Enrichment, Egress/Outbound, analytics, model application & Reporting
a. Passive - Data Discovery EDA (Exploratory Data Analytics)
(every layers ingestion/transformation/analytics/consumption) -
Performing an (Data Exploration) exploratory data analysis of the raw data to identify the attributes and patterns.
b. Active - Combining Data + Schema Evolution/Merging (Structuring)
c. Validation, Cleansing, Scrubbing - Identifying and filling gaps & Cleaning data to remove outliers and inaccuracies
Preprocessing, Preparation
Cleansing (removal of unwanted datasets eg. na.drop),
Scrubbing (convert of raw to tidy na.fill or na.replace),
d. Standardization, De Duplication and Replacement & Deletion of Data to make it in a usable format (Dataengineers/consumers)

2. Data Enrichment - Makes your data rich and detailed
a. Add, Remove, Rename, Modify/replace
b. split, merge/Concat
c. Type Casting, format & Schema Migration

3. Data Customization & Processing - Application of Tailored Business specific Rules
a. User Defined Functions
b. Building of Frameworks & Reusable Functions

4. Data Curation
a. Curation/Transformation
b. Analysis/Analytics & Summarization -> filter, transformation, Grouping, Aggregation/Summarization

5. Data Wrangling - Gathering, Enriching and Transfomation of pre processed data into usable data
a. Lookup/Reference
b. Enrichment
c. Joins
d. Sorting
e. Windowing, Statistical & Analytical processing
f. Set Operation

6. Data Publishing & Consumption - Enablement of the Cleansed, transformed and analysed data as a Data Product.
a. Discovery,
b. Outbound/Egress,
c. Reports/exports
d. Schema migration
'''